In [ ]:
# Part B: Person Following with Obstacle Avoidance
# This implementation builds upon Tutorial 5 (YOLO detection), Tutorial 2 (camera setup),
# and Tutorial 4 (traitlets pattern) to create a robust person-following system.

"""
STEP 1: Display Widgets Setup
Based on Tutorial 2 Part A - standard widget configuration for side-by-side display
"""
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images (Tutorial 2 pattern)
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg for displaying (Tutorial 2)
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])


"""
STEP 2: Load YOLO Model
Based on Tutorial 5 - using TensorRT engine for real-time human detection
"""
from ultralytics import YOLO

# Load the YOLO11 TensorRT model (Tutorial 5)
model = YOLO("yolo11l_half.engine")


"""
STEP 3: Person Re-Identification System
Uses HSV histogram matching to track the same person across frames,
even when temporarily occluded by other people (handles occlusion requirement)
"""
import numpy as np

class PersonTracker:
    """
    Tracks a specific person using HSV histogram matching.
    This handles the "occlusion by other people" requirement.
    
    How it works:
    1. When target is locked, store their HSV histogram
    2. For each detection, compare histogram similarity
    3. Best match above threshold is considered the same person
    4. This allows re-identification after temporary occlusion
    """
    
    def __init__(self, similarity_threshold=0.6, h_bins=16, s_bins=16):
        self.target_histogram = None  # Stores target person's appearance
        self.target_locked = False
        self.similarity_threshold = similarity_threshold
        self.h_bins = h_bins
        self.s_bins = s_bins
        self.frames_lost = 0  # Track how long target has been lost
        self.max_lost_frames = 30  # Give up after 30 frames (~1 second at 30fps)
    
    def compute_histogram(self, roi_bgr):
        """
        Compute normalized HSV histogram for a region of interest.
        HSV is better than RGB for tracking because it's more robust to lighting changes.
        """
        if roi_bgr.size == 0:
            return None
        
        # Convert to HSV color space (more robust to lighting)
        hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
        
        # Calculate histogram using Hue and Saturation channels
        # Ranges: H=[0,180], S=[0,256]
        hist = cv2.calcHist(
            [hsv], [0, 1], None,
            [self.h_bins, self.s_bins],
            [0, 180, 0, 256]
        )
        
        # Normalize histogram to [0, 1] range
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        return hist
    
    def lock_target(self, frame, bbox):
        """
        Lock onto a specific person by storing their histogram.
        Called when we first select our target person.
        """
        x1, y1, x2, y2 = map(int, bbox)
        roi = frame[y1:y2, x1:x2]
        
        self.target_histogram = self.compute_histogram(roi)
        if self.target_histogram is not None:
            self.target_locked = True
            self.frames_lost = 0
            return True
        return False
    
    def find_target(self, frame, detections):
        """
        Find the target person among all detected people.
        Returns (found, bbox, distance) tuple.
        
        Uses histogram matching to identify the same person even after occlusion.
        """
        if not self.target_locked:
            return False, None, float('inf')
        
        best_match_score = -1
        best_bbox = None
        best_distance = float('inf')
        
        # Compare each detected person to our target histogram
        for detection in detections:
            bbox, distance = detection
            x1, y1, x2, y2 = map(int, bbox)
            roi = frame[y1:y2, x1:x2]
            
            hist = self.compute_histogram(roi)
            if hist is None:
                continue
            
            # Calculate histogram similarity using correlation
            # Returns value in [-1, 1], where 1 = perfect match
            similarity = cv2.compareHist(
                self.target_histogram, 
                hist, 
                cv2.HISTCMP_CORREL
            )
            
            if similarity > best_match_score:
                best_match_score = similarity
                best_bbox = bbox
                best_distance = distance
        
        # Check if best match is good enough
        if best_match_score > self.similarity_threshold:
            self.frames_lost = 0
            return True, best_bbox, best_distance
        else:
            # Target not found in this frame
            self.frames_lost += 1
            if self.frames_lost > self.max_lost_frames:
                # Lost target for too long, unlock
                self.target_locked = False
            return False, None, float('inf')
    
    def reset(self):
        """Reset tracker to allow locking onto a new person."""
        self.target_histogram = None
        self.target_locked = False
        self.frames_lost = 0


"""
STEP 4: Human Detection and Distance Calculation
Based on Tutorial 5 - YOLO detection with depth measurement
"""

def detect_all_humans(frame, depth_image, conf_threshold=0.5):
    """
    Detect all humans in frame and calculate their distances.
    Returns list of (bbox, distance) tuples.
    
    Based on Tutorial 5:
    - YOLO detects humans (class 0)
    - For each detection, get bounding box
    - Calculate center point
    - Read depth at center to get distance
    """
    results = model(frame, verbose=False)
    
    detections = []
    
    for result in results:
        for i in range(len(result.boxes.cls)):
            # Only process human detections (class 0)
            if result.boxes.cls[i] == 0:
                if result.boxes.conf[i] > conf_threshold:
                    # Get bounding box coordinates [x1, y1, x2, y2]
                    bbox = result.boxes.xyxy[i].cpu().numpy()
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get distance from depth image at center point
                    # Tutorial 2 explains depth image usage
                    if (0 <= center_y < depth_image.shape[0] and 
                        0 <= center_x < depth_image.shape[1]):
                        distance = depth_image[center_y, center_x]
                        
                        # Filter out invalid depth readings
                        if not np.isnan(distance) and distance > 0:
                            detections.append((bbox, distance))
    
    return detections


"""
STEP 5: Obstacle Detection Using YOLO
Detects objects in central region that could block path to target.
Uses YOLO for semantic understanding - distinguishes people from objects.
"""

def detect_obstacles_semantic(frame, depth_image, target_distance, conf_threshold=0.5):
    """
    Detect obstacles (people and objects) that are closer than target person.
    This implements semantic obstacle detection - we know WHAT the obstacle is.
    
    Returns (obstacle_detected, obstacle_side, closest_distance)
    - obstacle_side: 'left', 'right', or 'center'
    """
    results = model(frame, verbose=False)
    
    frame_center_x = frame.shape[1] / 2
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    
    # Define central region where obstacles matter (similar to Tutorial 3 Part B)
    # We only care about obstacles in the robot's path
    left_boundary = frame_width * 0.25   # 25% from left
    right_boundary = frame_width * 0.75  # 75% from left
    top_boundary = frame_height * 0.25   # 25% from top
    
    closest_obstacle_distance = float('inf')
    obstacle_side = 'center'
    obstacle_detected = False
    
    for result in results:
        for i in range(len(result.boxes.cls)):
            # Process all objects (people are class 0, other objects are other classes)
            if result.boxes.conf[i] > conf_threshold:
                bbox = result.boxes.xyxy[i].cpu().numpy()
                
                # Calculate center and check if in central region
                center_x = int((bbox[0] + bbox[2]) / 2)
                center_y = int((bbox[1] + bbox[3]) / 2)
                
                # Only consider objects in our path (central region and below top boundary)
                if (left_boundary <= center_x <= right_boundary and 
                    center_y > top_boundary):
                    
                    # Get distance
                    if (0 <= center_y < depth_image.shape[0] and 
                        0 <= center_x < depth_image.shape[1]):
                        distance = depth_image[center_y, center_x]
                        
                        # Check if obstacle is closer than target
                        # And closer than our minimum safe distance
                        if (not np.isnan(distance) and 
                            distance > 0 and 
                            distance < target_distance and 
                            distance < 800):  # 800mm minimum clearance
                            
                            if distance < closest_obstacle_distance:
                                closest_obstacle_distance = distance
                                obstacle_detected = True
                                
                                # Determine which side the obstacle is on
                                if center_x < frame_center_x - 50:
                                    obstacle_side = 'left'
                                elif center_x > frame_center_x + 50:
                                    obstacle_side = 'right'
                                else:
                                    obstacle_side = 'center'
    
    return obstacle_detected, obstacle_side, closest_obstacle_distance


"""
STEP 6: Camera System Initialization
Based on Tutorial 2 Part B and Tutorial 4 Part A - threaded camera capture with traitlets
"""
import traitlets
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize robot motor control (Tutorial 1)
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    """
    Camera class using traitlets for real-time frame monitoring.
    Based on Tutorial 4 Part A pattern - separates data capture from processing.
    """
    color_value = traitlets.Any()  # Monitored variable for frame updates
    
    def __init__(self):
        super(Camera, self).__init__()
        
        # Initialize ZED camera (Tutorial 2 Part B)
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA      # Best depth quality
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Distance in mm
        
        # Open camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)
        
        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False
        
        # Get camera resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        
        # Allocate memory for images
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
    
    def _capture_frames(self):
        """
        Threaded frame capture loop (Tutorial 4 Part A pattern).
        Runs continuously in background, updating images.
        """
        while self.thread_runnning_flag:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve color and depth images
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                
                # Convert BGRA to BGR for OpenCV compatibility
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                
                # Get depth as numpy array
                self.depth_image = np.asanyarray(self.depth.get_data())
    
    def start(self):
        """Start the camera capture thread."""
        if not self.thread_runnning_flag:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()
    
    def stop(self):
        """Stop the camera capture thread and motors."""
        if self.thread_runnning_flag:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

# Initialize and start camera
camera = Camera()
camera.start()


"""
STEP 7: Following Behavior with Differential Drive
Uses smooth curved movement with individual wheel speed control.
Based on the cosine approach from love_detection.ipynb.
"""

def follow_target(frame, target_bbox, target_distance, depth_colormap):
    """
    Follow the target person while maintaining optimal distance.
    Uses differential drive for smooth curved paths.
    
    Distance thresholds:
    - < 600mm: Too close, back up
    - 600-900mm: Optimal distance, stop
    - 900-1500mm: Medium range, approach slowly
    - > 1500mm: Far away, approach faster
    """
    # Calculate target position relative to frame center
    bbox_center_x = (target_bbox[0] + target_bbox[2]) / 2
    frame_center_x = camera.width / 2
    
    # Calculate angle offset (normalized to -1 to 1)
    offset_x = bbox_center_x - frame_center_x
    max_offset = camera.width / 2
    normalized_angle = offset_x / max_offset  # -1 (left) to 1 (right)
    
    # Convert to radians for differential drive calculation
    angle_rad = normalized_angle * (np.pi / 4)  # Max ±π/4 for smooth turning
    
    # Calculate differential drive speeds using cosine
    # This creates smooth curved paths instead of discrete turns
    turning_factor = np.cos(angle_rad)
    
    # Draw target bounding box
    cv2.rectangle(frame, 
                  (int(target_bbox[0]), int(target_bbox[1])), 
                  (int(target_bbox[2]), int(target_bbox[3])), 
                  (0, 255, 0), 3)  # Green box for target
    
    # Distance-based behavior
    if target_distance < 600:
        # Too close - back up
        robot.backward(0.2)
        cv2.putText(depth_colormap, f'TOO CLOSE {target_distance:.0f}mm', 
                   (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
    elif 600 <= target_distance <= 900:
        # Optimal distance - stop and maintain
        robot.stop()
        cv2.putText(depth_colormap, f'OPTIMAL {target_distance:.0f}mm', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
    elif 900 < target_distance <= 1500:
        # Medium distance - approach with differential steering
        base_speed = 0.3
        
        if angle_rad > 0.1:  # Target on right
            left_speed = base_speed
            right_speed = base_speed * turning_factor
            robot.frontLeft(speed=left_speed)
            robot.backLeft(speed=left_speed)
            robot.frontRight(speed=right_speed)
            robot.backRight(speed=right_speed)
            cv2.putText(depth_colormap, f'FOLLOW RIGHT {target_distance:.0f}mm', 
                       (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
            
        elif angle_rad < -0.1:  # Target on left
            left_speed = base_speed * turning_factor
            right_speed = base_speed
            robot.frontLeft(speed=left_speed)
            robot.backLeft(speed=left_speed)
            robot.frontRight(speed=right_speed)
            robot.backRight(speed=right_speed)
            cv2.putText(depth_colormap, f'FOLLOW LEFT {target_distance:.0f}mm', 
                       (160, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
            
        else:  # Target centered
            robot.forward(base_speed)
            cv2.putText(depth_colormap, f'APPROACH {target_distance:.0f}mm', 
                       (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
            
    else:  # target_distance > 1500
        # Far away - approach faster
        base_speed = 0.5
        
        if angle_rad > 0.1:  # Target on right
            left_speed = base_speed
            right_speed = base_speed * turning_factor
            robot.frontLeft(speed=left_speed)
            robot.backLeft(speed=left_speed)
            robot.frontRight(speed=right_speed)
            robot.backRight(speed=right_speed)
            cv2.putText(depth_colormap, f'CHASE RIGHT {target_distance:.0f}mm', 
                       (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
            
        elif angle_rad < -0.1:  # Target on left
            left_speed = base_speed * turning_factor
            right_speed = base_speed
            robot.frontLeft(speed=left_speed)
            robot.backLeft(speed=left_speed)
            robot.frontRight(speed=right_speed)
            robot.backRight(speed=right_speed)
            cv2.putText(depth_colormap, f'CHASE LEFT {target_distance:.0f}mm', 
                       (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
            
        else:  # Target centered
            robot.forward(base_speed)
            cv2.putText(depth_colormap, f'CHASE {target_distance:.0f}mm', 
                       (220, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)


"""
STEP 8: Obstacle Avoidance Behavior
When obstacle blocks path, navigate around it while trying to maintain
visual contact with target person.
"""

def avoid_obstacle(obstacle_side, depth_colormap):
    """
    Navigate around obstacle based on which side it's on.
    Simple strategy: turn away from obstacle until path is clear.
    
    Args:
        obstacle_side: 'left', 'right', or 'center'
    """
    if obstacle_side == 'left':
        # Obstacle on left, turn right
        robot.right(0.3)
        cv2.putText(depth_colormap, 'OBSTACLE LEFT - TURNING RIGHT', 
                   (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
    elif obstacle_side == 'right':
        # Obstacle on right, turn left
        robot.left(0.3)
        cv2.putText(depth_colormap, 'OBSTACLE RIGHT - TURNING LEFT', 
                   (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
    else:  # center
        # Obstacle directly ahead, pick a direction (prefer right)
        robot.right(0.3)
        cv2.putText(depth_colormap, 'OBSTACLE AHEAD - TURNING RIGHT', 
                   (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)


"""
STEP 9: Main Control Loop with Keyboard Input
Based on Answer_keyboard_control.ipynb pattern
"""

# Initialize tracker
tracker = PersonTracker(similarity_threshold=0.6)

# State machine states
current_state = 'idle'  # States: 'idle', 'locking', 'following', 'avoiding'

# Create keyboard input widget (Answer_keyboard_control.ipynb pattern)
text_input = widgets.Text(
    value='',
    placeholder='Type: l=Lock target, s=Stop, r=Reset',
    description='Control:',
    disabled=False
)

def on_text_change(change):
    """Handle keyboard input commands."""
    global current_state
    
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()
        
        if last_char == 'l':
            # Lock onto target
            current_state = 'locking'
            print('\nLOCKING - Will lock onto closest person')
            
        elif last_char == 's':
            # Stop everything
            current_state = 'idle'
            tracker.reset()
            robot.stop()
            print('\nSTOPPED - Robot halted, tracker reset')
            
        elif last_char == 'r':
            # Reset tracker only
            tracker.reset()
            current_state = 'idle'
            robot.stop()
            print('\nRESET - Tracker reset, ready to lock new target')

text_input.observe(on_text_change, names='value')
display(text_input)


def main_callback(change):
    """
    Main control callback - executed every frame when color_value changes.
    Implements state machine for target locking, following, and obstacle avoidance.
    """
    global current_state
    
    # Get current frame
    frame = change['new'].copy()
    depth_image = camera.depth_image
    
    # Create depth colormap for visualization
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(depth_image, alpha=0.03), 
        cv2.COLORMAP_JET
    )
    
    # Detect all humans in frame (Tutorial 5 method)
    all_detections = detect_all_humans(frame, depth_image)
    
    # Draw all detected humans in blue
    for bbox, distance in all_detections:
        cv2.rectangle(frame, 
                     (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), 
                     (255, 0, 0), 2)  # Blue boxes for all humans
    
    # State machine
    if current_state == 'idle':
        robot.stop()
        cv2.putText(frame, 'IDLE - Press L to lock target', 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(depth_colormap, 'IDLE', 
                   (270, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
    elif current_state == 'locking':
        # Lock onto closest detected person
        if len(all_detections) > 0:
            # Find closest person
            closest_detection = min(all_detections, key=lambda x: x[1])
            closest_bbox, closest_distance = closest_detection
            
            # Lock onto this person
            if tracker.lock_target(frame, closest_bbox):
                current_state = 'following'
                print(f'\nTARGET LOCKED at {closest_distance:.0f}mm')
                
                # Draw green box around locked target
                cv2.rectangle(frame, 
                            (int(closest_bbox[0]), int(closest_bbox[1])), 
                            (int(closest_bbox[2]), int(closest_bbox[3])), 
                            (0, 255, 0), 3)
        else:
            cv2.putText(frame, 'LOCKING - No person detected', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
    
    elif current_state == 'following':
        # Try to find our target person
        target_found, target_bbox, target_distance = tracker.find_target(frame, all_detections)
        
        if target_found:
            # Check for obstacles in our path
            obstacle_detected, obstacle_side, obstacle_distance = detect_obstacles_semantic(
                frame, depth_image, target_distance
            )
            
            if obstacle_detected:
                # Obstacle blocking path - switch to avoiding
                current_state = 'avoiding'
                cv2.putText(frame, 'AVOIDING OBSTACLE', 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                # Path clear - follow target
                follow_target(frame, target_bbox, target_distance, depth_colormap)
                cv2.putText(frame, f'FOLLOWING - Distance: {target_distance:.0f}mm', 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Lost target
            robot.stop()
            if tracker.frames_lost > tracker.max_lost_frames:
                # Lost for too long, give up
                current_state = 'idle'
                print('\nTARGET LOST - Returning to idle')
            cv2.putText(frame, f'SEARCHING - Target lost for {tracker.frames_lost} frames', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
            cv2.putText(depth_colormap, 'TARGET LOST', 
                       (220, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
    
    elif current_state == 'avoiding':
        # Try to find target to check if we should resume following
        target_found, target_bbox, target_distance = tracker.find_target(frame, all_detections)
        
        if target_found:
            # Check if obstacle is still there
            obstacle_detected, obstacle_side, obstacle_distance = detect_obstacles_semantic(
                frame, depth_image, target_distance
            )
            
            if obstacle_detected:
                # Still blocked - keep avoiding
                avoid_obstacle(obstacle_side, depth_colormap)
                cv2.putText(frame, 'AVOIDING OBSTACLE', 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                # Path clear - resume following
                current_state = 'following'
                print('\nPath clear - Resuming following')
        else:
            # Lost target while avoiding
            robot.stop()
            current_state = 'idle'
            print('\nTarget lost during avoidance - Returning to idle')
    
    # Display status information
    cv2.putText(frame, f'State: {current_state.upper()}', 
               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.putText(frame, f'Tracked: {tracker.target_locked}', 
               (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Display images
    scale = 0.15
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

# Attach callback to camera (Tutorial 4 Part A pattern)
camera.observe(main_callback, names=['color_value'])

print('\n=== Part B: Person Following System ===')
print('Commands:')
print('  l = Lock onto closest person and start following')
print('  s = Stop robot and reset tracker')
print('  r = Reset tracker only (keeps running)')
print('\nFeatures:')
print('  - Automatic person re-identification after occlusion')
print('  - Obstacle avoidance while following')
print('  - Smooth curved movement with differential drive')
print('  - Maintains optimal distance (600-900mm)')
print('========================================')